In [1]:
from dataloader import TFDNA_ds
import torch

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
# @@@@@@@@ DNA PATH?!?!
data = TFDNA_ds(TF_path='final_tf_data.h5', DNA_path='?')

N_train = int(len(data)*0.9)

training_dataset = torch.utils.data.Subset(data,range(N_train))
validation_dataset = torch.utils.data.Subset(data,range(N_train,len(data)))

In [ ]:
example, label = training_dataset[31]

In [ ]:
example, label

In [ ]:
from torch.utils.data import Dataset, DataLoader

batch_size = 100

data_loader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True)
validation_data_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
for batched_e, l in data_loader:
    break

In [ ]:
batched_e, l

In [ ]:
from model import EncoderNet

In [ ]:
n_layers = 6
d_model = 50
n_heads = 4
dropout_enc = 0.1
window_size = 300
dropout_emb = True

net = EncoderNet(n_layers=n_layers, d_model=d_model, n_heads=n_heads, dropout_enc=dropout_enc, window_size=window_size, dropout_emb=dropout_emb)

In [ ]:
net(batched_e).shape

In [ ]:
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda:0')

net.to(device)

In [ ]:
import torch.nn as nn
import torch.optim as optim

# @@@@@@@@ WHAT LOSS FUNCTION?!?!?
loss_func = nn.KLDivLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0005) 

In [ ]:
def compute_accuracy_and_loss(dataloader, net):
    total = 0
    correct = 0
    loss = 0
    
    if torch.cuda.is_available():
        net.cuda()
    net.eval()
    
    n_batches = 0
    with torch.no_grad():
        for batched_e, l in dataloader:
            n_batches += 1

            if torch.cuda.is_available():
                batched_e = batched_e.to(torch.device('cuda'))
                l = l.cuda()
            pred = net(batched_e)         

            loss += loss_func(pred, l).item()
            #pred = torch.argmax(pred,dim=1)
            #correct += len(torch.where(pred==l)[0])
            #total += len(l)
   
    loss = loss / n_batches      
    return loss

In [ ]:
from tqdm.notebook import tqdm

n_epochs = 5
bestacc  = -999

training_loss_vs_epoch = []
validation_loss_vs_epoch = []

training_acc_vs_epoch = []
validation_acc_vs_epoch = []

pbar = tqdm( range(n_epochs) )

for epoch in pbar:
    
    if len(validation_loss_vs_epoch) > 1:
        pbar.set_description('val acc:'+'{0:.5f}'.format(validation_acc_vs_epoch[-1])+', train acc:'+'{0:.5f}'.format(training_acc_vs_epoch[-1]))
       
    net.train() # put the net into "training mode"
    for batched_e, l in data_loader:
        if torch.cuda.is_available():
            batched_g = batched_e.to(torch.device('cuda'))
            l = l.cuda()
            
        optimizer.zero_grad()
        pred = net(batched_e)
        loss = loss_func(pred,l)
        loss.backward()
        optimizer.step()
    
    net.eval() #put the net into evaluation mode
    train_acc, train_loss = compute_accuracy_and_loss(data_loader, net)
    valid_acc, valid_loss =  compute_accuracy_and_loss(validation_data_loader, net)
         
    training_loss_vs_epoch.append(train_loss)    
    training_acc_vs_epoch.append( train_acc )
    
    validation_acc_vs_epoch.append(valid_acc)
    validation_loss_vs_epoch.append(valid_loss)

    #keep model with best accuracy
    if len(validation_loss_vs_epoch)==1 or validation_acc_vs_epoch[-1] > bestacc:
        torch.save(net.state_dict(), 'trained_model.pt')
        bestacc = validation_acc_vs_epoch[-1]

In [ ]:
print(bestacc)

In [ ]:
import matplotlib.pyplot as plt

fig,ax = plt.subplots(1,2,figsize=(8,3))

ax[0].plot(training_loss_vs_epoch,label='training')
ax[0].plot(validation_loss_vs_epoch,label='validation')
ax[0].set_title('Loss')
ax[0].legend()

ax[1].plot(training_acc_vs_epoch,label='training')
ax[1].plot(validation_acc_vs_epoch,label='validation')
ax[1].set_title('Accuracy')
ax[1].legend()

plt.show()